In [2]:


%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') 


def read_vocabulary(file_path):
    return set(word.strip() for word in open(file_path))

stop_words = read_vocabulary("stop_words_en.txt")

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        if word in stop_words:
            print >> sys.stderr, "reporter:counter:Wiki stats,Stop words,%d" % 1            
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)



Overwriting mapper.py


In [7]:

%%writefile counter_process.py

import re
import sys
def read_vocabulary(file_path):
    return set(word.strip() for word in open(file_path))

lines = read_vocabulary(sys.argv[1])

Total_words = 0
Stop_words = 0
for line in lines:
    if "Total words=" in line:
        Total_words = int(re.search(r'\d+', line).group(0))
        
    if "Stop words=" in line:
        Stop_words = int(re.search(r'\d+', line).group(0))

print(Stop_words / float(Total_words) * 100)


Overwriting counter_process.py


In [4]:
%%bash

OUT_DIR="wordcount_result_stopwords"
NUM_REDUCERS=0
LOGS="stderr_logs.txt"

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming stopwords" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> $LOGS
    
python ./counter_process.py $LOGS

cat $LOGS >&2

38.44036900909957


rm: `wordcount_result_1581421827075344': No such file or directory
20/02/11 11:50:31 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/02/11 11:50:32 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/02/11 11:50:33 INFO mapred.FileInputFormat: Total input files to process : 1
20/02/11 11:50:33 INFO mapreduce.JobSubmitter: number of splits:2
20/02/11 11:50:34 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1581420788885_0001
20/02/11 11:50:34 INFO impl.YarnClientImpl: Submitted application application_1581420788885_0001
20/02/11 11:50:35 INFO mapreduce.Job: The url to track the job: http://1b3d269503ff:8088/proxy/application_1581420788885_0001/
20/02/11 11:50:35 INFO mapreduce.Job: Running job: job_1581420788885_0001
20/02/11 11:50:44 INFO mapreduce.Job: Job job_1581420788885_0001 running in uber mode : false
20/02/11 11:50:44 INFO mapreduce.Job:  map 0% reduce 0%
20/02/11 11:51:02 INFO mapreduce.Job:  map 16% reduce 0%
20/02/11 11:5